## Collecting the Data for training the model.

In [1]:
import spacy

import re
nlp = spacy.load('en')
file = open('/home/tanmay/Desktop/NLP Workflow/Working with a big data/collected_data.txt','r')
minidata = file.read()
club = [text.lower() for text in re.findall('[A-Za-z]+',minidata)]

## Initializing The tokenizer

In [2]:
from keras.preprocessing.text import Tokenizer

tk = Tokenizer(num_words=len(club), lower= True)
tk.fit_on_texts(club)
sequence = tk.texts_to_sequences(club)

## Building up the matrix from raw data with the help of thread

In [3]:
import math
training_length =math.trunc(math.sqrt(len(sequence)))
import threading

class thread(threading.Thread):
    def __init__(self, threadID, name):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
    def run(self, sequence):
        new_Matrix = []
        for i in range(training_length,len(sequence)):
            row = sequence[i-training_length:i]
            new_Matrix.append(row)
        return new_Matrix

new_Matrix = thread(1, "Thread-1").run(sequence)

import numpy as np
Matrix = np.array(new_Matrix)
print(Matrix.shape)
vocabulary_size =len(club)

## Saparating the Matrix in X and y

In [4]:
from keras.utils import to_categorical
X=Matrix[:,:-1]
y =Matrix[:,-1]
y = to_categorical(y, num_classes= vocabulary_size +1)
seq_len = X.shape[1]

## The model Creating function

In [5]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from tensorflow.python.keras.backend import categorical_crossentropy

def create_model(vocab_Size, seq_Len):
    model = Sequential()
    model.add(Embedding(vocab_Size,seq_Len,input_length=seq_Len))
    model.add(LSTM(seq_Len*2,return_sequences=True))
    model.add(LSTM(seq_Len*2))
    model.add(Dense(seq_Len*2,activation='relu'))
    model.add(Dense(vocab_Size,activation = 'softmax'))

    model.compile(loss ='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])
    print("Summary :-")
    print(model.summary())
    return model

model = create_model(vocabulary_size+ 1, seq_len)


Summary :-
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 7, 7)              490       
_________________________________________________________________
lstm (LSTM)                  (None, 7, 14)             1232      
_________________________________________________________________
lstm_1 (LSTM)                (None, 14)                1624      
_________________________________________________________________
dense (Dense)                (None, 14)                210       
_________________________________________________________________
dense_1 (Dense)              (None, 70)                1050      
Total params: 4,606
Trainable params: 4,606
Non-trainable params: 0
_________________________________________________________________
None


## Training and saving the value of X, y and Model itself.

In [8]:
from pickle import dump
model.fit(X,y,batch_size = 128, verbose=2, epochs=3)

model.save('model.h5')

pickle_out = open('X.pickle','wb')
dump(X,pickle_out)
pickle_out.close()
pickle_out = open('y.pickle','wb')
dump(y,pickle_out)
pickle_out.close()


Epoch 1/3
1/1 - 0s - loss: 4.2419 - accuracy: 0.0656
Epoch 2/3
1/1 - 0s - loss: 4.2408 - accuracy: 0.0656
Epoch 3/3
1/1 - 0s - loss: 4.2396 - accuracy: 0.0656
